In [15]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import RandomForestRegressor

from sklearn import cross_validation

import numpy
import math

In [2]:
train_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Store,Date,,,,,,,
1,2015-07-31,5,5263,555,1,1,0,1
2,2015-07-31,5,6064,625,1,1,0,1
3,2015-07-31,5,8314,821,1,1,0,1
4,2015-07-31,5,13995,1498,1,1,0,1
5,2015-07-31,5,4822,559,1,1,0,1


In [6]:
train_df.reset_index(inplace=True)
features = [col for col in train_df.columns if col != 'Sales']

In [10]:

train_df['Date'] = train_df['Date'].map(lambda x: x.value)

In [9]:
def numericStateHoliday(x):
    if x == 'a':
        return 1
    elif x == 'b':
        return 2
    elif x == 'c':
        return 3
    else:
        return x
train_df['StateHoliday'] = train_df['StateHoliday'].map(lambda x: numericStateHoliday(x))

In [12]:
%%time
regressor = RandomForestRegressor(n_estimators=25, n_jobs=-1)
score = cross_validation.cross_val_score(regressor, train_df[features], train_df['Sales'], n_jobs=-1).mean()
print("Score: {}".format(score))

Score: 0.935614602253212
CPU times: user 848 ms, sys: 744 ms, total: 1.59 s
Wall time: 1min 23s


In [13]:
%%time
regressor = RandomForestRegressor(n_estimators=25, n_jobs=-1)
predict = cross_validation.cross_val_predict(regressor, train_df[features], train_df['Sales'], n_jobs=-1)

CPU times: user 760 ms, sys: 624 ms, total: 1.38 s
Wall time: 1min 23s


In [17]:

score_df = train_df[['Sales']]
score_df['Predict'] = predict
score_df = score_df[score_df['Sales'] != 0]
score_df['PercentError'] =  (score_df['Sales'] - score_df['Predict']) / score_df['Sales']
score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
math.sqrt(score_df['PercentErrorSquared'].mean())

/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


0.15026537992657213

In [23]:
import random
stores = random.sample(train_df['Store'].unique().tolist(), 10)

In [26]:

score_df = train_df[['Store', 'Sales']]
score_df['Predict'] = predict
score_df = score_df[score_df['Sales'] != 0]

/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [27]:

score_df['PercentError'] =  (score_df['Sales'] - score_df['Predict']) / score_df['Sales']
score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']

In [31]:
store_score_df = score_df.groupby("Store").mean()

In [33]:
store_score_df['RMSPE'] = numpy.sqrt(score_df['PercentErrorSquared'])

In [57]:
store_score_df.take(numpy.array(stores) - 1)['RMSPE']

Store
254     0.187299
1028    0.008113
125     0.207284
904     0.098605
436     0.143642
91      0.114540
1016    0.008613
699     0.049349
1036    0.092893
629     0.062789
Name: RMSPE, dtype: float64

In [77]:
def score(training_df, features):

    regressor = RandomForestRegressor(n_estimators=25, n_jobs=-1)
    predict = cross_validation.cross_val_predict(regressor, training_df[features], training_df['Sales'], n_jobs=-1)
    
    score_df = training_df[['Sales']].copy()
    score_df['Predict'] = predict
    score_df = score_df[score_df['Sales'] != 0]
    score_df['PercentError'] =  (score_df['Sales'] - score_df['Predict']) / score_df['Sales']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [58]:
scores = {}
for store in train_df['Store'].unique().tolist():
    store_df = train_df[train_df['Store'] == store]
    scores[store] = score(store_df)

scores

{1: 0.06177202836943382,
 2: 0.08023195039671856,
 3: 0.06829781439482115,
 4: 0.05086018147174186,
 5: 0.08082075926325966,
 6: 0.07469844350385743,
 7: 0.08675558102369826,
 8: 0.07474420206749992,
 9: 0.0638177211675068,
 10: 0.06782649764505203,
 11: 0.08086183777108374,
 12: 0.07486196389563046,
 13: 0.08291948739092682,
 14: 0.06016932763119164,
 15: 0.06889194265592873,
 16: 0.0759342165593759,
 17: 0.07515960141308972,
 18: 0.07269148530481487,
 19: 0.07347397879397759,
 20: 0.0737413907486127,
 21: 0.09478532707604684,
 22: 0.08324312661670304,
 23: 0.06751843488424486,
 24: 0.06940096509009337,
 25: 0.06689286560607927,
 26: 0.06433623009753078,
 27: 0.07116147291248802,
 28: 0.08906619943047488,
 29: 0.06674273362673049,
 30: 0.06963611737123618,
 31: 0.07469573852252791,
 32: 0.0701378044013875,
 33: 0.06621503630835629,
 34: 0.05553740599397912,
 35: 0.0695754336585419,
 36: 0.06824119317708567,
 37: 0.07053411087308352,
 38: 0.0735478422028153,
 39: 0.08167743925206496,
 

In [65]:
numpy.mean(numpy.array(list(scores.values())))

0.078116490828677276

In [66]:
max(numpy.array(list(scores.values())))

0.9135807027904872

In [71]:
train, test = cross_validation.train_test_split(train_df, test_size=.1)

In [72]:
regressor_all = RandomForestRegressor(n_estimators=25, n_jobs=-1)
regressor_all.fit(train[features], train['Sales'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=25, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [73]:
predictions_all_test = regressor_all.predict(test[features])

In [74]:
predict = cross_validation.cross_val_predict(RandomForestRegressor(n_estimators=25, n_jobs=-1), train[features], train['Sales'], n_jobs=-1)

In [76]:
train['BigModel-Sales'] = predict

/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [86]:
scores = {}

features = [col for col in train.columns if col != 'Sales'] # Pick up bigModel-Sales
i = 0
for store in train['Store'].unique().tolist():
    if (i % 100 == 0):
        print (i)
    i = i + 1
    store_df = train[train['Store'] == store]
    scores[store] = score(store_df, features)

scores

0
100
200
300
400
500
600
700
800
900
1000
1100


{1: 0.06299535510837964,
 2: 0.08992634539183088,
 3: 0.06695744108763908,
 4: 0.0483968933855785,
 5: 0.0751796100288137,
 6: 0.0577149504126114,
 7: 0.06924681668801586,
 8: 0.07077884656633453,
 9: 0.059950948369831975,
 10: 0.061932666755705995,
 11: 0.0691279833564756,
 12: 0.06964867460805549,
 13: 0.07752928878552515,
 14: 0.05691866448891771,
 15: 0.06265169583995385,
 16: 0.06746249052220782,
 17: 0.06126679770432176,
 18: 0.0645199324055546,
 19: 0.07159298468983433,
 20: 0.064884660313526,
 21: 0.08734363866539893,
 22: 0.07381129192273009,
 23: 0.06254154143573128,
 24: 0.05977736971517243,
 25: 0.0774119360874237,
 26: 0.061013863526839955,
 27: 0.06658821566501893,
 28: 0.07610017229366756,
 29: 0.062408537904052465,
 30: 0.06425817209443833,
 31: 0.0643397725651085,
 32: 0.06884635361760282,
 33: 0.05833299268818261,
 34: 0.05220331298584465,
 35: 0.06859373874698213,
 36: 0.05895055261635597,
 37: 0.06702268759598033,
 38: 0.07203908060887938,
 39: 0.06711789292121335,


In [83]:
store_df = train[train_df['Store'] == store]

/opt/conda/lib/python3.4/site-packages/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [94]:
numpy.mean(numpy.array(list(scores.values())))

0.068843387673599779

In [110]:
%%time
regressors = {}

features = [col for col in train.columns if col != 'Sales'] # Pick up bigModel-Sales
i = 0
for store in train['Store'].unique().tolist():
    if (i % 100 == 0):
        print (i)
    i = i + 1
    store_df = train[train['Store'] == store]
    #scores[store] = score(store_df, features)
    
    regressor = RandomForestRegressor(n_estimators=25, n_jobs=-1)
    regressor.fit(store_df[features], store_df['Sales'])
    regressors[store] = regressor
    


0
100
200
300
400
500
600
700
800
900
1000
1100
CPU times: user 2min 3s, sys: 3.78 s, total: 2min 7s
Wall time: 2min 27s


In [103]:

features = [col for col in test.columns if col != 'Sales'] # Pick up bigModel-Sales
test['BigModel-Sales'] = regressor_all.predict(test[features])

/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [113]:

score = {}
features = [col for col in test.columns if col != 'Sales'] # Pick up bigModel-Sales
i = 0
for store, regressor in regressors.items():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_df = test[test['Store'] == store]
    #scores[store] = score(store_df, features)
    
    predict = regressor.predict(store_df[features])
    
    score_df = store_df[['Sales']].copy()
    score_df['Predict'] = predict
    score_df = score_df[score_df['Sales'] != 0]
    score_df['PercentError'] =  (score_df['Sales'] - score_df['Predict']) / score_df['Sales']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    score[store] = math.sqrt(score_df['PercentErrorSquared'].mean())
    


0
100
200
300
400
500
600
700
800
900
1000
1100


In [118]:
numpy.mean(numpy.array(list(score.values())))

0.064177212781291557

In [117]:
store_df

,Store,Date,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,BigModel-Sales
682709,1115,1382918400000000000,1,3784,315,1,0,0,0,4046.60
214079,1115,1421798400000000000,3,4000,318,1,0,0,0,4222.92
672674,1115,1383696000000000000,3,5612,384,1,1,0,0,5625.48
561174,1115,1392336000000000000,5,5966,437,1,0,0,0,6220.12
31219,1115,1435968000000000000,6,6543,466,1,0,0,0,6321.76
990449,1115,1359072000000000000,5,5082,373,1,1,0,0,5654.12
964804,1115,1361059200000000000,7,0,0,0,0,0,0,0.00
631419,1115,1386892800000000000,5,6276,441,1,0,0,0,6245.08
994909,1115,1358726400000000000,1,3723,243,1,1,0,0,3265.80
729539,1115,1379289600000000000,1,3664,309,1,0,0,0,4058.68


# Submission


In [130]:
test_df = DataFrame.from_csv("test.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
test_df.reset_index(inplace=True)
test_df['Date'] = test_df['Date'].map(lambda x: x.value)

def numericStateHoliday(x):
    if x == 'a':
        return 1
    elif x == 'b':
        return 2
    elif x == 'c':
        return 3
    else:
        return x
test_df['StateHoliday'] = test_df['StateHoliday'].map(lambda x: int(numericStateHoliday(x)))

In [131]:
test_df.dtypes

Store              int64
Date               int64
Id                 int64
DayOfWeek          int64
Open             float64
Promo              int64
StateHoliday       int64
SchoolHoliday      int64
dtype: object

In [138]:
test_df.head()

,Store,Date,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
0,1,1442448000000000000,1,4,1,1,0,0
1,3,1442448000000000000,2,4,1,1,0,0
2,7,1442448000000000000,3,4,1,1,0,0
3,8,1442448000000000000,4,4,1,1,0,0
4,9,1442448000000000000,5,4,1,1,0,0


In [136]:
test

,Store,Date,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,BigModel-Sales
938757,713,1363046400000000000,2,6537,836,1,0,0,0,6890.68
743378,459,1378166400000000000,2,8087,1153,1,0,0,0,8182.32
15085,591,1437177600000000000,6,4429,372,1,0,0,0,4459.44
379498,75,1406764800000000000,4,8449,796,1,1,0,1,8330.44
375670,1085,1407196800000000000,2,8545,823,1,1,0,1,9038.08
94267,608,1431043200000000000,5,10110,1035,1,1,0,0,9452.72
588274,340,1390176000000000000,1,10671,981,1,1,0,0,10453.04
678138,1004,1383264000000000000,5,0,0,0,0,1,0,0.00
301199,366,1414022400000000000,4,6737,652,1,1,0,0,6535.20
198311,957,1423008000000000000,3,8525,889,1,1,0,1,8275.24


In [135]:

features = [col for col in test_df.columns if col != 'Sales' and col != 'Promo'] # Pick up bigModel-Sales
test_df['BigModel-Sales'] = regressor_all.predict(test_df[features])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:

score = {}
features = [col for col in test.columns if col != 'Sales'] # Pick up bigModel-Sales
i = 0
for store, regressor in regressors.items():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_df = test[test['Store'] == store]
    #scores[store] = score(store_df, features)
    
    predict = regressor.predict(store_df[features])
    store_df['Prediction']